In [3]:
import pandas as pd 
import tensorflow as tf 
#import matplotlib.pyplot as plt 
import numpy as np 

In [17]:
required=['loc','d_hub']
df=pd.read_csv('train_data_full.csv',usecols=required)
#print(df)
#df=df['loc','d_hub']
df=df.dropna()

coor_x=df['loc'].str.extract(r'.*\[(.*)\,.*')
coor_y=df['loc'].str.extract(r'.*\,(.*)\].*')
#for i in df['loc']:
    #spilter=i.split(',')
    #coor_x.append(float(spilter[0].strip()))
    #coor_y.append(float(spilter[1]))
print(coor_x)
print(coor_y)


                        0
0               28.464536
1              28.4476783
2             28.45060518
3       28.46705306941655
4       28.47559833333333
5       28.46705306941655
6              28.4859103
7             28.50427754
8              28.4512372
9      28.472815947120495
10      28.47204029560089
11            28.41409992
12             28.4640685
13             28.4620959
14            28.48661114
15             28.5062985
16             28.4675452
17             28.4617113
18     28.475516438484192
19      28.45555333706083
20             28.4710968
21            28.51102671
22             28.4571634
23             28.4607675
24               28.40495
25             28.4332475
26      28.45366333333333
27              28.381413
28     28.481469999999998
29             28.4593564
...                   ...
17248         28.48517662
17249   28.41730166666667
17250          28.4865608
17251          28.4783526
17252          28.4624405
17253         28.37275394
17254       

In [18]:
list_of_hubs=[]

for i in df['d_hub']:
	if i not in list_of_hubs:
		list_of_hubs.append(i) 


#print("number of unique d_hubs:"+str(len(list_of_hubs)))
size=len(list_of_hubs)

#coor_x=[]
#coor_y=[]

#print(df['loc'])

#df['loc']=df['loc'].str.extract(r'.*\[(.*)\].*')
#print(df['loc'])

"""for i in df['loc']:
	splitter=[]
	splitter=i.split(',')
	coor_x.append(float(splitter[0]))
	coor_y.append(float(splitter[1]))
print("coor_x:")
print(coor_x)
print("coor_y:")
print(coor_y)"""
list_d=[]
for i in df['d_hub']:
	list_d.append(list_of_hubs.index(i))

df['d']=list_d
df['coor_x']=coor_x
df['coor_y']=coor_y
del df['loc']
del df['d_hub']
df[['coor_x','coor_y']]=df[['coor_x','coor_y']].apply(pd.to_numeric)

    


print(df)


"""plt.scatter(df['coor_x'],df['coor_y'])
plt.show()"""

input_x=df.loc[:,["coor_x","coor_y"]].values
input_y=df.loc[:,["d"]].values
#test_x=df.loc[55:63,["coor_x","coor_y"]].values
#test_y=df.loc[55:63,["d"]].values

       d     coor_x     coor_y
0      0  28.464536  77.023254
1      1  28.447678  77.093965
2      0  28.450605  77.035139
3      0  28.467053  77.033728
4      1  28.475598  77.078655
5      0  28.467053  77.033728
6      0  28.485910  77.019509
7      0  28.504278  77.029119
8      1  28.451237  77.091520
9      0  28.472816  77.010409
10     0  28.472040  77.051663
11     2  28.414100  77.068842
12     0  28.464068  77.025018
13     0  28.462096  77.012113
14     1  28.486611  77.062654
15     0  28.506298  77.041642
16     1  28.467545  77.081104
17     0  28.461711  77.045088
18     0  28.475516  77.048680
19     3  28.455553  77.050544
20     0  28.471097  77.048667
21     3  28.511027  77.045385
22     1  28.457163  77.094271
23     0  28.460767  77.043815
24     4  28.404950  77.047321
25     4  28.433247  77.046923
26     1  28.453663  77.096207
27     4  28.381413  77.060157
28     0  28.481470  77.046990
29     1  28.459356  77.093405
...   ..        ...        ...
17248  1

In [19]:
#LEARNING HERE
x=tf.placeholder(tf.float32,[None,2])
y=tf.placeholder(tf.float32,[None,1])

#layer 1
w1=tf.Variable(tf.truncated_normal(shape=[2,100],stddev=0.1))
b1=tf.Variable(tf.zeros([100]))
h1=tf.nn.relu(tf.add(tf.matmul(x,w1),b1))
#w=tf.Variable(tf.zeros([2,size]))
#b=tf.Variable(tf.zeros([size]))
#layer 2
w2=tf.Variable(tf.truncated_normal(shape=[100,60],stddev=0.1))
b2=tf.Variable(tf.zeros([60]))
h2=tf.nn.relu(tf.matmul(h1,w2)+b2)

#layer 3
w3=tf.Variable(tf.truncated_normal(shape=[60,20],stddev=0.1))
b3=tf.Variable(tf.zeros([20]))
h3=tf.nn.relu(tf.matmul(h2,w3)+b3)



w4=tf.Variable(tf.truncated_normal(shape=[20,size],stddev=0.1))
b4=tf.Variable(tf.zeros([size]))


predicted=tf.add(tf.matmul(h3,w4),b4)
#curve_handle=tf.reciprocal(1+tf.exp(-predicted))


In [20]:


learning_rate=0.01
training_epochs=25000
display_steps=1000
n_samples=input_y.size

#cost=tf.reduce_sum(tf.pow(y-predicted,2))/(2*n_samples)
y_=tf.reciprocal(1 + tf.exp(-predicted))
cost=tf.reduce_sum(tf.pow(y-y_,2))/(n_samples)
optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)



In [21]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

for i in range(training_epochs):
    sess.run(optimizer,feed_dict={x:input_x,y:input_y})
    if i%display_steps==0:
        c=sess.run(cost,feed_dict={x:input_x,y:input_y})
        print("training ",'%04d'%i," cost=","{:.09f}".format(c))
print("OPTIMIZATION IS DONE!!!")

training  0000  cost= nan


KeyboardInterrupt: 

In [35]:
"""correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(sess.run(accuracy,feed_dict={x:input_x,y_:input_y}))"""

'correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))\naccuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))\nprint(sess.run(accuracy,feed_dict={x:input_x,y_:input_y}))'

In [65]:

print("training cost=",sess.run(cost,feed_dict={x:input_x,y:input_y}),"\nW=",sess.run(w4),"\nb=",sess.run(b1))


"""a=(input("x-coor:"))
b=tf.float32(input("y-coor:"))
#a=np.array([[a, b]],dtype=np.float32)
a=[a, b]
print(a)"""

training cost= 7.177099 
W= [[-0.03793808  0.09199846  0.1558005  -0.01959503 -0.04706092]
 [-0.06999644 -0.05643506  0.06280132 -0.04207415  0.15978093]
 [ 0.16838859  0.05080476  0.03467909  0.15029585  0.22075339]
 [-0.06717242  0.00079426 -0.02195795  0.01296374  0.05452399]
 [ 0.09751768 -0.02235989 -0.06154672 -0.12152424  0.05152001]
 [-0.07009715  0.12005881 -0.04656963  0.06798658 -0.01952686]
 [-0.10757761 -0.11678332  0.05184102 -0.11997538 -0.16812895]
 [ 0.05766153  0.04972052  0.05860519 -0.02660579  0.10204935]
 [-0.00117337 -0.13017254  0.01765574  0.05476211  0.05219555]
 [-0.16122892 -0.00817725 -0.08474181  0.063665    0.02490867]
 [ 0.09179594  0.03607472  0.01793083  0.12763348 -0.00332362]
 [-0.19137749  0.00190957 -0.05524538  0.01955752 -0.12332239]
 [ 0.1183577   0.17668782 -0.05899192 -0.10055691  0.01274543]
 [ 0.1034179   0.20261478  0.26213536  0.16538905  0.07551894]
 [-0.04581017 -0.1427408   0.04811118  0.12240756  0.06622022]
 [-0.12985629  0.01819491 -

'a=(input("x-coor:"))\nb=tf.float32(input("y-coor:"))\n#a=np.array([[a, b]],dtype=np.float32)\na=[a, b]\nprint(a)'

In [66]:
print(list_of_hubs)
print( sess.run(y_, feed_dict={x: input_x}))

['b1b3de810f1d46619cd868a470ab831c', '48b6dba5107b422fb17325c85021dd4f', '550789510924496fb451b653cabc626f', 'a67a898ec4f4482bb79ee39293c330e9', 'be7261b61da149aa805e10f395c46ed3']
[[0.8617697  0.860897   0.8591162  0.8613093  0.8624789 ]
 [0.86454284 0.8636242  0.8617568  0.86406773 0.86527586]
 [0.8629064  0.8620139  0.86019546 0.86243945 0.86362594]
 [0.861881   0.8610069  0.8592234  0.8614203  0.862591  ]
 [0.86249286 0.8616099  0.8598106  0.86202973 0.8632073 ]
 [0.861881   0.8610069  0.8592234  0.8614203  0.862591  ]
 [0.8603781  0.8595305  0.85779756 0.8599262  0.86107427]
 [0.85950035 0.8586693  0.8569681  0.8590543  0.86018777]
 [0.8642708  0.8633568  0.8614984  0.8637973  0.8650014 ]
 [0.8609442  0.86008584 0.85833204 0.8604885  0.861646  ]
 [0.86203057 0.86115456 0.8593681  0.86156946 0.8627415 ]
 [0.86591035 0.8649665  0.86305046 0.86542636 0.86665654]
 [0.86184216 0.8609683  0.8591853  0.8613815  0.8625521 ]
 [0.861637   0.8607662  0.8589887  0.8611771  0.86234534]
 [0.861

In [144]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
print(correct_prediction)

Tensor("Equal_4:0", shape=(?,), dtype=bool)


In [145]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(accuracy)

Tensor("Mean_5:0", shape=(), dtype=float32)


In [146]:
print(sess.run(accuracy,feed_dict={x:test_x,y:test_y}))

0.0


In [103]:
z=np.array([[28.4710968,77.0486671]])
print(z)

[[28.4710968 77.0486671]]
